## https://medium.com/@ansabiqbal/delta-lake-introduction-with-examples-using-pyspark-cb2a0d7a549d

### https://docs.delta.io/latest/delta-intro.html

In [3]:
#!pip install delta-spark
#!pip show delta-spark
#!pip list

In [1]:
#  Create a spark session with Delta

import pyspark
from delta import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

#help(delta)

if SparkSession.getActiveSession() == None:
    print("Active spark session not found")
else:
    print("Active spark session found")

builder = (
    pyspark.sql.SparkSession.builder
    #.master('local') # local- no parallelizm at all, local[2] - 2 cores, local[*] - as many cores as local logical cores
    .appName("DeltaTutorial") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")  # to allow deltatable.vacuum(0)
)

# Create spark context
spark = configure_spark_with_delta_pip(builder).getOrCreate() # function of the delta module
spark.sparkContext.setLogLevel("ERROR")

# check spark-warehouse databases & tables
print("from SQL:")
spark.sql("Show databases").show()
spark.sql("select current_database()").show()
#spark.sql("use mytestdb")
spark.sql("Show tables").show()
spark.sql("Show views").show()

# same as
print("from python:")
print(spark.catalog.listDatabases())
print(spark.catalog.currentDatabase())
#spark.catalog.setCurrentDatabase('mytestdb')
print(spark.catalog.listTables())

#spark.catalog.listColumns('t1')

print("Done.")

Active spark session not found
:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6fa9ffd-3a73-4c32-86b3-06523758251b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 194ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

from SQL:
+---------+
|namespace|
+---------+
|  default|
+---------+

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
+---------+--------+-----------+

from python:
[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/jovyan/work/various_tests/spark/delta_tests/spark-warehouse')]
default
[]
Done.


In [2]:
# get the size (in bytes) of a directory

import os

def get_size(path: str) -> int:    
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
                
    print(f"Size({path}): {total_size}")

get_size("data/delta-table")

Size(data/delta-table): 41060


In [3]:
# Create a spark dataframe and write as a delta table to an UNMANAGED!! database (file system folder)
import random
import shutil

df_data = []
rows = 100
print(f"Generating data ({rows} rows)...")
for c in range(rows):
    df_data.append([
        c,
        None if random.random() < 0.1 else "firstname-" + str(random.randint(1, 20)).zfill(3), 
        None if random.random() < 0.1 else "lastname-" + str(random.randint(1, 20)).zfill(3), 
        None if random.random() < 0.1 else "house-" + str(random.randint(1, 20)).zfill(3), 
        None if random.random() < 0.1 else "location-" + str(random.randint(1, 20)).zfill(3), 
        None if random.random() < 0.1 else random.randint(1, 99)
        ])

df_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("house", StringType(), True),
    StructField("location", StringType(), True),
    StructField("age", IntegerType(), True)
])

# drop any delta table files...
print("dropping old table files if exists...")
shutil.rmtree("data/delta-table", ignore_errors=True)

print("creating delta table...")
df = spark.createDataFrame(data=df_data, schema=df_schema)
df.write.mode(saveMode="overwrite").format("delta").save("data/delta-table")  # would go to managed DB if saveAsTable(...) is used!
   
print("Done.")

Generating data (100 rows)...
dropping old table files if exists...
creating delta table...


Done.


In [116]:
# Read data as df (not as DeltaTable) from UNMANAGED!! database (file system folder)
print("Reading delta file as df...")
df = spark.read.format("delta").load("data/delta-table")
df.orderBy("id").show(5)

Reading delta file as df...
+---+-------------+------------+---------+------------+---+
| id|    firstname|    lastname|    house|    location|age|
+---+-------------+------------+---------+------------+---+
|  0|firstname-015|lastname-018|house-017|location-002| 22|
|  1|firstname-016|lastname-006|house-011|location-001|  2|
|  2|firstname-015|lastname-013|house-010|        NULL| 77|
|  3|firstname-001|lastname-003|house-011|location-015| 55|
|  4|         NULL|lastname-002|house-002|location-008| 59|
+---+-------------+------------+---------+------------+---+
only showing top 5 rows



In [5]:
# Update data in Delta table, UNMANAGED!! database (file system folder)

# read delta table to deltaTable object directly, then converting to df
print("Reading delta file as delta object...")
deltaTable = DeltaTable.forPath(spark, "data/delta-table")
print("Converting delta object to df...")
deltaTable.toDF().show(5)

get_size("data/delta-table")

print("Updating delta table (object)...")
deltaTable.update(
    condition = expr("id = 0"),  # sql style
    #condition = expr("firstname = 'firstname-018'"),  # sql style
    #condition = expr("firstname is null"), # sql style
    #condition = col("firstname").isNull(), # spark style
    #set = {"lastname": lit("Hello7"), "house": lit("hello8")}
    #set = {"age": expr("age + 1000")} # lit('literal"), col('colname"), expr(col(colname)+'hello")
    set = {"house": expr("concat('new_', house)")}  # must use concat instead of "+"!!
        )

get_size("data/delta-table")

deltaTable.toDF().filter("id = 0 or firstname == 'firstname-018' or firstname is null").orderBy("id").show(5)
print("Done.")

Reading delta file as delta object...
Converting delta object to df...


+---+-------------+------------+-------------+------------+---+
| id|    firstname|    lastname|        house|    location|age|
+---+-------------+------------+-------------+------------+---+
|  0|firstname-015|lastname-017|new_house-002|location-012| 84|
|  1|firstname-017|        NULL|    house-017|location-017| 80|
|  2|firstname-009|lastname-004|    house-009|location-003| 81|
|  3|firstname-002|lastname-015|    house-020|        NULL|  8|
|  4|firstname-011|lastname-001|    house-019|location-002| 20|
+---+-------------+------------+-------------+------------+---+
only showing top 5 rows

Size(data/delta-table): 23904
Updating delta table (object)...


Size(data/delta-table): 41060
+---+-------------+------------+-----------------+------------+---+
| id|    firstname|    lastname|            house|    location|age|
+---+-------------+------------+-----------------+------------+---+
|  0|firstname-015|lastname-017|new_new_house-002|location-012| 84|
| 33|         NULL|lastname-003|    new_house-015|location-002| 12|
| 44|         NULL|lastname-002|    new_house-014|location-008| 63|
| 53|firstname-018|lastname-016|    new_house-006|location-016| 34|
| 55|         NULL|lastname-012|    new_house-010|location-020| 51|
+---+-------------+------------+-----------------+------------+---+
only showing top 5 rows

Done.


In [118]:
# merge with SQL syntax using UNMANAGED!! database (file system folder)
spark.sql("CREATE OR REPLACE TEMPORARY VIEW newData AS SELECT col1 AS id FROM VALUES 1,3,5,7,9,11,13,15,17,19")

# get the delta table to update
deltaTable = DeltaTable.forPath(spark, "data/delta-table")
deltaTable.toDF().createOrReplaceTempView("t1")  # df --> temp table

# do the merge
spark.sql("""
MERGE INTO t1 AS oldData
USING newData
ON oldData.id = newData.id
WHEN MATCHED
  THEN UPDATE SET house = concat('new', house)
WHEN NOT MATCHED
  THEN INSERT (id) VALUES (newData.id)
"""
)

deltaTable.toDF().orderBy("id").show(5)
print("Done.")

+---+-------------+------------+-------------+------------+---+
| id|    firstname|    lastname|        house|    location|age|
+---+-------------+------------+-------------+------------+---+
|  0|firstname-015|lastname-018|new_house-017|location-002| 22|
|  1|firstname-016|lastname-006| newhouse-011|location-001|  2|
|  2|firstname-015|lastname-013|    house-010|        NULL| 77|
|  3|firstname-001|lastname-003| newhouse-011|location-015| 55|
|  4|         NULL|lastname-002|    house-002|location-008| 59|
+---+-------------+------------+-------------+------------+---+
only showing top 5 rows

Done.


In [119]:
# merge with DF syntax, using UNMANAGED!! database (file system folder)
print("Upserting delta...")
# delta table path
deltaTable = DeltaTable.forPath(spark, "data/delta-table")
deltaTable.toDF().orderBy("id").show(5)

# define new data
df_data = [(99,"Gendry", "Baratheon", "Baratheon", "Kings Landing", 19),
        (126,"Jon", "Snow", "Stark", "Winterfell", 21),
        (127,"Jamie", "Lannister", "Lannister", "Casterly Rock", 36)
        ]

df_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("house", StringType(), True),
    StructField("location", StringType(), True),
    StructField("age", IntegerType(), True)
])

newData = spark.createDataFrame(data=df_data, schema=df_schema)

deltaTable.alias("oldData") \
    .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
    .whenMatchedUpdate(
    set={"firstname": col("newData.firstname"), "lastname": col("newData.lastname"), "house": col("newData.house"),
         "location": col("newData.location"), "age": col("newData.age")}) \
    .whenNotMatchedInsert(
    values={"id": col("newData.id"), "firstname": col("newData.firstname"), "lastname": col("newData.lastname"), "house": col("newData.house"),
            "location": col("newData.location"), "age": col("newData.age")}) \
    .execute()

deltaTable.toDF().orderBy(desc("id")).show(10)
print("Done.")

Upserting delta...
+---+-------------+------------+-------------+------------+---+
| id|    firstname|    lastname|        house|    location|age|
+---+-------------+------------+-------------+------------+---+
|  0|firstname-015|lastname-018|new_house-017|location-002| 22|
|  1|firstname-016|lastname-006| newhouse-011|location-001|  2|
|  2|firstname-015|lastname-013|    house-010|        NULL| 77|
|  3|firstname-001|lastname-003| newhouse-011|location-015| 55|
|  4|         NULL|lastname-002|    house-002|location-008| 59|
+---+-------------+------------+-------------+------------+---+
only showing top 5 rows

+---+-------------+------------+---------+-------------+----+
| id|    firstname|    lastname|    house|     location| age|
+---+-------------+------------+---------+-------------+----+
|127|        Jamie|   Lannister|Lannister|Casterly Rock|  36|
|126|          Jon|        Snow|    Stark|   Winterfell|  21|
| 99|       Gendry|   Baratheon|Baratheon|Kings Landing|  19|
| 98|fir

In [120]:
# Delete data (single record) using UNMANAGED!! database (file system folder)
print("Deleting data...")

# delta table path
deltaTable = DeltaTable.forPath(spark, "data/delta-table")
deltaTable.toDF().orderBy(asc("id")).show()

deltaTable.delete(condition=expr("id in (0)"))

deltaTable.toDF().orderBy(asc("id")).show(10)
print("Done.")

Deleting data...
+---+-------------+------------+-------------+------------+----+
| id|    firstname|    lastname|        house|    location| age|
+---+-------------+------------+-------------+------------+----+
|  0|firstname-015|lastname-018|new_house-017|location-002|  22|
|  1|firstname-016|lastname-006| newhouse-011|location-001|   2|
|  2|firstname-015|lastname-013|    house-010|        NULL|  77|
|  3|firstname-001|lastname-003| newhouse-011|location-015|  55|
|  4|         NULL|lastname-002|    house-002|location-008|  59|
|  5|firstname-001|lastname-012| newhouse-005|location-013|  29|
|  6|firstname-002|lastname-003|    house-015|location-002|  59|
|  7|firstname-010|lastname-009| newhouse-020|location-006|  21|
|  8|firstname-011|lastname-010|    house-008|location-020|  74|
|  9|firstname-020|lastname-013| newhouse-011|location-009|  31|
| 10|firstname-013|lastname-012|    house-003|location-003|  24|
| 11|firstname-001|lastname-010| newhouse-012|location-001|NULL|
| 12|fir

In [6]:
# Reading delta details, history, all versions of delta table using UNMANAGED!! database (file system folder)
# deleting a single record creates a new parquet file and "removes" the old one, but still keeps the old one for history
# i.e. delta format is just a metadata management of the basic parquet format!!!

def print_dict(d: dict, level: int = 0) -> None:

    sp = "".ljust(level * 3)
    for k in d.keys():
        print(f"{sp}{k}: {d[k]}")
        
        if type(d[k]) is dict:
            print_dict(d[k], level + 1)
            
    if level == 0:
        print()

deltaTable = DeltaTable.forPath(spark, "data/delta-table")

print("Reading delta table info from deltaTable.details()...")
df_info = deltaTable.detail().select("*").collect() # returns list of Row
print_dict(df_info[0].asDict())

print("Reading delta table history from deltaTable.history()...")
df_hist = deltaTable.history()
for row in df_hist.select("*").orderBy("version", ascending = 1).collect():
    print_dict(row.asDict())
    
    df = spark.read.format("delta").option("versionAsOf", row['version']).load("data/delta-table").filter("id <= 5").show()

Reading delta table info from deltaTable.details()...


format: delta
id: e6434d4e-2951-4560-a29d-c17fb59fc930
name: None
description: None
location: file:/home/jovyan/work/various_tests/spark/delta_tests/data/delta-table
createdAt: 2025-05-12 14:11:21.415000
lastModified: 2025-05-12 14:21:11.020000
partitionColumns: []
clusteringColumns: []
numFiles: 4
sizeInBytes: 9805
properties: {}
minReaderVersion: 1
minWriterVersion: 2
tableFeatures: ['appendOnly', 'invariants']

Reading delta table history from deltaTable.history()...
version: 0
timestamp: 2025-05-12 14:11:25.722000
userId: None
userName: None
operation: WRITE
operationParameters: {'mode': 'Overwrite', 'partitionBy': '[]'}
   mode: Overwrite
   partitionBy: []
job: None
notebook: None
clusterId: None
readVersion: None
isolationLevel: Serializable
isBlindAppend: False
operationMetrics: {'numOutputRows': '100', 'numOutputBytes': '9730', 'numFiles': '4'}
   numOutputRows: 100
   numOutputBytes: 9730
   numFiles: 4
userMetadata: None
engineInfo: Apache-Spark/3.5.5 Delta-Lake/3.3.0



+---+-------------+------------+---------+------------+---+
| id|    firstname|    lastname|    house|    location|age|
+---+-------------+------------+---------+------------+---+
|  0|firstname-015|lastname-017|house-002|location-012| 84|
|  1|firstname-017|        NULL|house-017|location-017| 80|
|  2|firstname-009|lastname-004|house-009|location-003| 81|
|  3|firstname-002|lastname-015|house-020|        NULL|  8|
|  4|firstname-011|lastname-001|house-019|location-002| 20|
|  5|firstname-016|lastname-004|house-004|location-007| 91|
+---+-------------+------------+---------+------------+---+

version: 1
timestamp: 2025-05-12 14:17:55.307000
userId: None
userName: None
operation: UPDATE
operationParameters: {'predicate': '["(id#830 = 0)"]'}
   predicate: ["(id#830 = 0)"]
job: None
notebook: None
clusterId: None
readVersion: 0
isolationLevel: Serializable
isBlindAppend: False
operationMetrics: {'numDeletionVectorsUpdated': '0', 'numAddedFiles': '1', 'executionTimeMs': '1426', 'numDeleti

+---+-------------+------------+-------------+------------+---+
| id|    firstname|    lastname|        house|    location|age|
+---+-------------+------------+-------------+------------+---+
|  0|firstname-015|lastname-017|new_house-002|location-012| 84|
|  1|firstname-017|        NULL|    house-017|location-017| 80|
|  2|firstname-009|lastname-004|    house-009|location-003| 81|
|  3|firstname-002|lastname-015|    house-020|        NULL|  8|
|  4|firstname-011|lastname-001|    house-019|location-002| 20|
|  5|firstname-016|lastname-004|    house-004|location-007| 91|
+---+-------------+------------+-------------+------------+---+

version: 2
timestamp: 2025-05-12 14:21:11.020000
userId: None
userName: None
operation: UPDATE
operationParameters: {'predicate': '[]'}
   predicate: []
job: None
notebook: None
clusterId: None
readVersion: 1
isolationLevel: Serializable
isBlindAppend: False
operationMetrics: {'numDeletionVectorsUpdated': '0', 'numAddedFiles': '4', 'executionTimeMs': '931',

In [122]:
# delete all history
# first set spark config "spark.databricks.delta.retentionDurationCheck.enabled" = false if retentionHours is < 168
# vacuum() has a *BUG*, it deletes a file from the oldest version, so ruins the entire table....

deltaTable = DeltaTable.forPath(spark, "data/delta-table")
#deltaTable.vacuum(15/60)
print("Done.")

Done.


In [123]:
# "timetravel" query as of timestamp / as of version using MANAGED!! database (spark-warehouse)
# would go to unmanaged DB if USING csv OPTIONS PATH '...' is used
import time

# create
spark.sql("""
CREATE TABLE IF NOT EXISTS t2 (
  id INT,
  name STRING
)
USING DELTA
""")

# insert
spark.sql("""
INSERT INTO  t2 
select 0,'hello0' union all 
select 1,'hello1'
""")

# insert OVERWRITE (old parquet overwritten)
spark.sql("""
INSERT OVERWRITE t2 
select 2,'hello0' union all 
select 3,'hello1'
""")

# wait for as of timestamp query
print("waiting for 5 secs...")
time.sleep(5)

# update
spark.sql("""
UPDATE t2
SET t2.name = concat(t2.name, ' ', 'xxx')
WHERE t2.id = 0
""")

# get hist
spark.sql("""
DESCRIBE HISTORY t2
""").select("version", "timestamp").show(truncate=False)

# select
spark.sql("""
SELECT * FROM t2
""").show()

# select from a specific version
spark.sql("""
SELECT * FROM t2 VERSION AS OF 1
""").show()

# select from a specific timestamp (must be between min/max history.  "describe history" cannot be queried...)
table_name = "t2"
spark.sql(f"DESCRIBE HISTORY {table_name}").select("version", "timestamp").show(truncate=False)
min_ts = spark.sql(f"DESCRIBE HISTORY {table_name}").select("timestamp").orderBy("version", ascending = 1).collect()[0]["timestamp"]
max_ts = spark.sql(f"DESCRIBE HISTORY {table_name}").select("timestamp").orderBy("version", ascending = 0).collect()[0]["timestamp"]
print(min_ts, max_ts)
spark.sql(f"""SELECT * FROM t2 TIMESTAMP AS OF '{min_ts}'""").show()
spark.sql(f"""SELECT * FROM t2 TIMESTAMP AS OF '{max_ts}'""").show()

# restore to a previous version
spark.sql("""
RESTORE t2 VERSION AS OF 1
""").show()

# drop
spark.sql("""
DROP TABLE t2
""")

print("Done.")

waiting for 5 secs...
+-------+-----------------------+
|version|timestamp              |
+-------+-----------------------+
|2      |2025-04-01 19:56:02.729|
|1      |2025-04-01 19:56:01.509|
|0      |2025-04-01 19:56:01.309|
+-------+-----------------------+

+---+------+
| id|  name|
+---+------+
|  2|hello0|
|  3|hello1|
+---+------+

+---+------+
| id|  name|
+---+------+
|  1|hello1|
|  0|hello0|
+---+------+

+-------+-----------------------+
|version|timestamp              |
+-------+-----------------------+
|2      |2025-04-01 19:56:02.729|
|1      |2025-04-01 19:56:01.509|
|0      |2025-04-01 19:56:01.309|
+-------+-----------------------+

2025-04-01 19:56:01.309000 2025-04-01 19:56:02.729000
+---+----+
| id|name|
+---+----+
+---+----+

+---+------+
| id|  name|
+---+------+
|  2|hello0|
|  3|hello1|
+---+------+



+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|table_size_after_restore|num_of_files_after_restore|num_removed_files|num_restored_files|removed_files_size|restored_files_size|
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|                    1402|                         2|                2|                 2|              1402|               1402|
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+

Done.


In [124]:
# create a delta table through SQL syntax (USING DELTA)

spark.sql("Show databases").show()
spark.sql("select current_database()").show()
#spark.sql("use mytestdb")
spark.sql("Show tables").show()
spark.sql("Show views").show()

# create
spark.sql("""
CREATE TABLE IF NOT EXISTS my_table (
  id INT,
  name STRING
)
USING DELTA
""")

# insert
spark.sql("""
INSERT INTO  my_table values (1,'hello1')
""")


# update
spark.sql("""
UPDATE my_table t1
SET t1.name = concat(t1.name, ' ', 'xxx')
WHERE t1.id < 100
""").show()

# select
spark.sql("""
SELECT * FROM my_table
""").show()

# drop
spark.sql("""
DROP TABLE my_table
""").show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|       t1|      false|
|         |  newdata|       true|
|         |       t1|       true|
+---------+---------+-----------+

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
|         | newdata|       true|
|         |      t1|       true|
+---------+--------+-----------+

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+

+---+----------+
| id|      name|
+---+----------+
|  1|hello1 xxx|
+---+----------+

++
||
++
++



In [27]:
# create a view on an unmanaged delta file
spark.sql("""
create or replace temporary view v1
using delta
options(path 'data/delta-table')
""").show()

spark.sql("""select * from v1""").show(5, truncate=False)

spark.sql("Show databases").show()
spark.sql("select current_database()").show()
#spark.sql("use mytestdb")
spark.sql("Show tables").show()
spark.sql("Show views").show()

++
||
++
++

+---+-------------+------------+---------+------------+----+
|id |firstname    |lastname    |house    |location    |age |
+---+-------------+------------+---------+------------+----+
|32 |firstname-003|lastname-017|house-018|location-017|3   |
|33 |NULL         |lastname-015|house-006|location-003|71  |
|34 |firstname-009|lastname-017|house-001|location-016|51  |
|35 |firstname-007|lastname-006|house-003|location-018|NULL|
|36 |firstname-010|lastname-005|house-006|location-002|44  |
+---+-------------+------------+---------+------------+----+
only showing top 5 rows

+---------+
|namespace|
+---------+
|  default|
+---------+

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |       v1|       true|
+---------+---------+-----------+

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+--------